In [1]:
import re
import math
import warnings
warnings.filterwarnings("ignore")
import datetime
import operator
import numpy as np
import pandas as pd
import seaborn as sns
import datetime as dt
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
pd.set_option('display.max_columns', 550)
pd.set_option('display.max_rows', 550)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
# MARCH
df = pd.read_csv('C:/Users/zagkoui/OneDrive - Vodafone Group/Documents/database/March22/base_usage.csv', parse_dates= ["CONNECTION_DAY", "INSERTED"], sep='|')
demographics = pd.read_csv('C:/Users/zagkoui/OneDrive - Vodafone Group/Documents/database/March22/demographics.csv', sep='|')
consent = pd.read_csv('C:/Users/zagkoui/OneDrive - Vodafone Group/Documents/database/March22/consent.csv', sep='|')
post_code = pd.read_excel('C:/Users/zagkoui/OneDrive - Vodafone Group/Documents/database/March22/postcode_data.xlsx')
students = pd.read_csv('C:/Users/zagkoui/OneDrive - Vodafone Group/Documents/database/March22/students.csv', sep='|')

# APRIL
# df = pd.read_csv('C:/Users/zagkoui/OneDrive - Vodafone Group/Documents/database/Apr22/base_usage.csv', parse_dates= ["CONNECTION_DAY", "INSERTED"], sep='|')
# demographics = pd.read_csv('C:/Users/zagkoui/OneDrive - Vodafone Group/Documents/database/Apr22/demographics.csv', sep='|')
# consent = pd.read_csv('C:/Users/zagkoui/OneDrive - Vodafone Group/Documents/database/Apr22/consent.csv', sep='|')
# post_code = pd.read_excel('C:/Users/zagkoui/OneDrive - Vodafone Group/Documents/database/Apr22/postcode_data.xlsx')
# students = pd.read_csv('C:/Users/zagkoui/OneDrive - Vodafone Group/Documents/database/Apr22/students.csv', sep='|')

In [4]:
df.head()

,MSISDN,M1_TTL_OUT_CALLS,M1_TTL_OUT_MINUTES,M1_TTL_OUT_REVENUE,M1_V_CALLS_TO_FIXED_LINES,M1_V_MINUTES_TO_FIXED_LINES,M1_V_REVENUE_TO_FIXED_LINES,M1_V_CALLS_TO_COMPETITION,M1_V_MINUTES_TO_COMPETITION,M1_V_REVENUE_TO_COMPETITION,M1_V_CALLS_TO_INTERNATIONAL,M1_V_MINUTES_TO_INTERNATIONAL,M1_V_REVENUE_TO_INTERNATIONAL,M1_GPRS_SESSION,M1_GPRS_REVENUE,M1_GPRS_VOLUME,M1_RECHARGES_NUMBER,M1_RECHARGES_VALUE,M1_OUT_DAYS,M1_INC_DAYS,M1_BUNDLE_REVENUE,M2_TTL_OUT_CALLS,M2_TTL_OUT_MINUTES,M2_TTL_OUT_REVENUE,M2_V_CALLS_TO_FIXED_LINES,M2_V_MINUTES_TO_FIXED_LINES,M2_V_REVENUE_TO_FIXED_LINES,M2_V_CALLS_TO_COMPETITION,M2_V_MINUTES_TO_COMPETITION,M2_V_REVENUE_TO_COMPETITION,M2_V_CALLS_TO_INTERNATIONAL,M2_V_MINUTES_TO_INTERNATIONAL,M2_V_REVENUE_TO_INTERNATIONAL,M2_GPRS_SESSION,M2_GPRS_REVENUE,M2_GPRS_VOLUME,M2_RECHARGES_NUMBER,M2_RECHARGES_VALUE,M2_OUT_DAYS,M2_INC_DAYS,M2_BUNDLE_REVENUE,M3_TTL_OUT_CALLS,M3_TTL_OUT_MINUTES,M3_TTL_OUT_REVENUE,M3_V_CALLS_TO_FIXED_LINES,M3_V_MINUTES_TO_FIXED_LINES,M3_V_REVENUE_TO_FIXED_LINES,M3_V_CALLS_TO_COMPETITION,M3_V_MINUTES_TO_COMPETITION,M3_V_REVENUE_TO_COMPETITION,M3_V_CALLS_TO_INTERNATIONAL,M3_V_MINUTES_TO_INTERNATIONAL,M3_V_REVENUE_TO_INTERNATIONAL,M3_GPRS_SESSION,M3_GPRS_REVENUE,M3_GPRS_VOLUME,M3_RECHARGES_NUMBER,M3_RECHARGES_VALUE,M3_OUT_DAYS,M3_INC_DAYS,M3_BUNDLE_REVENUE,M4_TTL_OUT_CALLS,M4_TTL_OUT_MINUTES,M4_TTL_OUT_REVENUE,M4_V_CALLS_TO_FIXED_LINES,M4_V_MINUTES_TO_FIXED_LINES,M4_V_REVENUE_TO_FIXED_LINES,M4_V_CALLS_TO_COMPETITION,M4_V_MINUTES_TO_COMPETITION,M4_V_REVENUE_TO_COMPETITION,M4_V_CALLS_TO_INTERNATIONAL,M4_V_MINUTES_TO_INTERNATIONAL,M4_V_REVENUE_TO_INTERNATIONAL,M4_GPRS_SESSION,M4_GPRS_REVENUE,M4_GPRS_VOLUME,M4_RECHARGES_NUMBER,M4_RECHARGES_VALUE,M4_OUT_DAYS,M4_INC_DAYS,M4_BUNDLE_REVENUE,M5_TTL_OUT_CALLS,M5_TTL_OUT_MINUTES,M5_TTL_OUT_REVENUE,M5_V_CALLS_TO_FIXED_LINES,M5_V_MINUTES_TO_FIXED_LINES,M5_V_REVENUE_TO_FIXED_LINES,M5_V_CALLS_TO_COMPETITION,M5_V_MINUTES_TO_COMPETITION,M5_V_REVENUE_TO_COMPETITION,M5_V_CALLS_TO_INTERNATIONAL,M5_V_MINUTES_TO_INTERNATIONAL,M5_V_REVENUE_TO_INTERNATIONAL,M5_GPRS_SESSION,M5_GPRS_REVENUE,M5_RECHARGES_NUMBER,M5_RECHARGES_VALUE,M5_GPRS_VOLUME,M5_OUT_DAYS,M5_INC_DAYS,M5_BUNDLE_REVENUE,M6_TTL_OUT_CALLS,M6_TTL_OUT_MINUTES,M6_TTL_OUT_REVENUE,M6_V_CALLS_TO_FIXED_LINES,M6_V_MINUTES_TO_FIXED_LINES,M6_V_REVENUE_TO_FIXED_LINES,M6_V_CALLS_TO_COMPETITION,M6_V_MINUTES_TO_COMPETITION,M6_V_REVENUE_TO_COMPETITION,M6_V_CALLS_TO_INTERNATIONAL,M6_V_MINUTES_TO_INTERNATIONAL,M6_V_REVENUE_TO_INTERNATIONAL,M6_GPRS_SESSION,M6_GPRS_REVENUE,M6_GPRS_VOLUME,M6_RECHARGES_NUMBER,M6_RECHARGES_VALUE,M6_OUT_DAYS,M6_INC_DAYS,M6_BUNDLE_REVENUE,TARIFF_PLAN,CONNECTION_DAY,SMARTPHONE_FLAG,INSERTED
0,6948724826,119,3666,1.9192,15,1693,0.0,13,1689,0.0,0,0,0.0,181,0.0,1.377771e+06,1,10.91,22,14,10.0,105.0,2230.0,0.0000,8.0,835.0,0.0,7.0,1078.0,0.00,0.0,0.0,0.0,156.0,0.0,1.607826e+06,1.0,13.64,24.0,12.0,10.00,100.0,2271.0,0.6628,1.0,54.0,0.0,6.0,464.0,0.00,0.0,0.0,0.0,208.0,0.0,2.446356e+06,1.0,10.91,24.0,13.0,10.0,106.0,3334.0,0.2944,7.0,666.0,0.00,8.0,1230.0,0.0,0.0,0.0,0.0,169.0,0.0,2.596339e+06,1.0,10.71,25.0,15.0,10.0,119.0,1430.0,0.4424,6.0,404.0,0.0,4.0,600.0,0.00,0.0,0.0,0.0,209.0,0.0,1.0,10.71,2.069943e+06,25.0,14.0,10.0,148.0,12815.0,0.0,28.0,7163.0,0.0,16.0,2440.0,0.0,0.0,0.0,0.0,144.0,0.0,2.285379e+06,1.0,10.71,30.0,17.0,10.0,Cuba40,2010-03-24,Y,2022-03-31
1,6949281531,130,24590,0.9944,10,2663,0.0,103,19477,0.0,0,0,0.0,0,0.0,0.000000e+00,1,10.91,30,25,10.0,120.0,24784.0,4.8600,25.0,7720.0,0.0,84.0,15468.0,4.86,0.0,0.0,0.0,0.0,0.0,0.000000e+00,1.0,10.91,26.0,23.0,7.06,119.0,26501.0,0.0000,20.0,4451.0,0.0,80.0,17524.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000e+00,1.0,10.91,31.0,30.0,10.0,124.0,24413.0,0.9580,5.0,2137.0,0.81,94.0,18436.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,1.0,10.71,29.0,26.0,10.0,115.0,25779.0,1.0320,14.0,3025.0,0.0,76.0,18390.0,0.00,0.0,0.0,0.0,0.0,0.0,1.0,10.71,0.000000e+00,29.0,29.0,10.0,137.0,20040.0,0.0,21.0,2009.0,0.0,91.0,15261.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,1.0,10.71,31.

In [5]:
df.shape

(1025686, 125)

In [4]:
# MARCH
out_dir = 'C:/Users/zagkoui/OneDrive - Vodafone Group/Documents/HIGHER_BUNDLES/main_dataset_preprocessing/previous_final_db.csv'

#APRIL
#out_dir = 'C:/Users/zagkoui/OneDrive - Vodafone Group/Documents/HIGHER_BUNDLES/main_dataset_preprocessing/next_final_db.csv'

# ------------------------------------- BASE USAGE -----------------------------------------------------

In [4]:
# check if there are NaN values
df.isnull().values.any()
# 1. COUNT how many NaN values we have per each column
for entry in list(df.columns.values):
    if df[entry].isnull().sum() > 0:
        print(entry, ": ", df[entry].isnull().sum())
        # I have to check the proportion (null / total input)
        print("The proportion of null values is: ", ((df[entry].isnull().sum() / len(df[entry])) * 100, "%"))

True

M1_BUNDLE_REVENUE :  142227
The proportion of null values is:  (13.86652445290274, '%')
M2_TTL_OUT_CALLS :  4010
The proportion of null values is:  (0.3909578564979926, '%')
M2_TTL_OUT_MINUTES :  4010
The proportion of null values is:  (0.3909578564979926, '%')
M2_TTL_OUT_REVENUE :  4010
The proportion of null values is:  (0.3909578564979926, '%')
M2_V_CALLS_TO_FIXED_LINES :  4010
The proportion of null values is:  (0.3909578564979926, '%')
M2_V_MINUTES_TO_FIXED_LINES :  4010
The proportion of null values is:  (0.3909578564979926, '%')
M2_V_REVENUE_TO_FIXED_LINES :  4010
The proportion of null values is:  (0.3909578564979926, '%')
M2_V_CALLS_TO_COMPETITION :  4010
The proportion of null values is:  (0.3909578564979926, '%')
M2_V_MINUTES_TO_COMPETITION :  4010
The proportion of null values is:  (0.3909578564979926, '%')
M2_V_REVENUE_TO_COMPETITION :  4010
The proportion of null values is:  (0.3909578564979926, '%')
M2_V_CALLS_TO_INTERNATIONAL :  4010
The proportion of null values is:  (

The proportion of null values is:  (5.3514428392315, '%')
M6_OUT_DAYS :  54889
The proportion of null values is:  (5.3514428392315, '%')
M6_INC_DAYS :  54889
The proportion of null values is:  (5.3514428392315, '%')
M6_BUNDLE_REVENUE :  170192
The proportion of null values is:  (16.59299239728338, '%')


In [6]:
# Check categorical values if they already have a None-type value
for column in df.columns.values:
    if (df[column].dtype == 'object' or df[column].dtype == 'datetime64[ns]'):
        df[column].value_counts()

Cuba40    991597
Cuba       34676
Name: TARIFF_PLAN, dtype: int64

2021-05-11    1238
2021-10-29    1199
2021-09-11    1130
2019-10-31    1117
2021-04-11    1114
              ... 
1998-10-25       1
1998-09-13       1
2000-08-20       1
1998-03-25       1
2000-01-23       1
Name: CONNECTION_DAY, Length: 8816, dtype: int64

Y    819593
N    206680
Name: SMARTPHONE_FLAG, dtype: int64

2022-04-30    1026273
Name: INSERTED, dtype: int64

In [7]:
# Fill NaNs
for column in df.columns.values:
    if (df[column].dtype == 'int64' or df[column].dtype == 'float64'):
        df[column] = df[column].fillna(0)
    elif (df[column].dtype == 'object'):
        df[column] = df[column].fillna("N/A")
    elif (df[column].dtype == 'datetime64[ns]'):
        new_date =  dt.datetime(int(df[column].dt.year.mean()), int(df[column].dt.month.mean()) , int(df[column].dt.day.mean()))
        df[column] = df[column].fillna(new_date)

In [8]:
# check again for NaN values 
df.isnull().values.any()

False

# --------------------------------------------  CONVERT -------------------------------------------

In [9]:
# 5. Convert from second -> minutes
# 6. Convert from KByte -> MByte
for column in df.columns.values:
    if 'MINUTES' in column:
        df[column] = df[column] / 60
    if 'VOLUME' in column:
        df[column] = df[column] / 1024

# ------------------------------------- ADD ATTRIBUTES -----------------------------------------------

# AVERAGE TALK TIME PER CALL

In [10]:
# edw na vrw ton meso oro tou eksamhnou?
for month in range(1,7):
    df[f"M{month}_MINUTES_PER_CALL"] = df[f"M{month}_TTL_OUT_MINUTES"] / df[f"M{month}_TTL_OUT_CALLS"]

In [11]:
df.head()

,MSISDN,M1_TTL_OUT_CALLS,M1_TTL_OUT_MINUTES,M1_TTL_OUT_REVENUE,M1_V_CALLS_TO_FIXED_LINES,M1_V_MINUTES_TO_FIXED_LINES,M1_V_REVENUE_TO_FIXED_LINES,M1_V_CALLS_TO_COMPETITION,M1_V_MINUTES_TO_COMPETITION,M1_V_REVENUE_TO_COMPETITION,M1_V_CALLS_TO_INTERNATIONAL,M1_V_MINUTES_TO_INTERNATIONAL,M1_V_REVENUE_TO_INTERNATIONAL,M1_GPRS_SESSION,M1_GPRS_REVENUE,M1_GPRS_VOLUME,M1_RECHARGES_NUMBER,M1_RECHARGES_VALUE,M1_OUT_DAYS,M1_INC_DAYS,M1_BUNDLE_REVENUE,M2_TTL_OUT_CALLS,M2_TTL_OUT_MINUTES,M2_TTL_OUT_REVENUE,M2_V_CALLS_TO_FIXED_LINES,M2_V_MINUTES_TO_FIXED_LINES,M2_V_REVENUE_TO_FIXED_LINES,M2_V_CALLS_TO_COMPETITION,M2_V_MINUTES_TO_COMPETITION,M2_V_REVENUE_TO_COMPETITION,M2_V_CALLS_TO_INTERNATIONAL,M2_V_MINUTES_TO_INTERNATIONAL,M2_V_REVENUE_TO_INTERNATIONAL,M2_GPRS_SESSION,M2_GPRS_REVENUE,M2_GPRS_VOLUME,M2_RECHARGES_NUMBER,M2_RECHARGES_VALUE,M2_OUT_DAYS,M2_INC_DAYS,M2_BUNDLE_REVENUE,M3_TTL_OUT_CALLS,M3_TTL_OUT_MINUTES,M3_TTL_OUT_REVENUE,M3_V_CALLS_TO_FIXED_LINES,M3_V_MINUTES_TO_FIXED_LINES,M3_V_REVENUE_TO_FIXED_LINES,M3_V_CALLS_TO_COMPETITION,M3_V_MINUTES_TO_COMPETITION,M3_V_REVENUE_TO_COMPETITION,M3_V_CALLS_TO_INTERNATIONAL,M3_V_MINUTES_TO_INTERNATIONAL,M3_V_REVENUE_TO_INTERNATIONAL,M3_GPRS_SESSION,M3_GPRS_REVENUE,M3_GPRS_VOLUME,M3_RECHARGES_NUMBER,M3_RECHARGES_VALUE,M3_OUT_DAYS,M3_INC_DAYS,M3_BUNDLE_REVENUE,M4_TTL_OUT_CALLS,M4_TTL_OUT_MINUTES,M4_TTL_OUT_REVENUE,M4_V_CALLS_TO_FIXED_LINES,M4_V_MINUTES_TO_FIXED_LINES,M4_V_REVENUE_TO_FIXED_LINES,M4_V_CALLS_TO_COMPETITION,M4_V_MINUTES_TO_COMPETITION,M4_V_REVENUE_TO_COMPETITION,M4_V_CALLS_TO_INTERNATIONAL,M4_V_MINUTES_TO_INTERNATIONAL,M4_V_REVENUE_TO_INTERNATIONAL,M4_GPRS_SESSION,M4_GPRS_REVENUE,M4_GPRS_VOLUME,M4_RECHARGES_NUMBER,M4_RECHARGES_VALUE,M4_OUT_DAYS,M4_INC_DAYS,M4_BUNDLE_REVENUE,M5_TTL_OUT_CALLS,M5_TTL_OUT_MINUTES,M5_TTL_OUT_REVENUE,M5_V_CALLS_TO_FIXED_LINES,M5_V_MINUTES_TO_FIXED_LINES,M5_V_REVENUE_TO_FIXED_LINES,M5_V_CALLS_TO_COMPETITION,M5_V_MINUTES_TO_COMPETITION,M5_V_REVENUE_TO_COMPETITION,M5_V_CALLS_TO_INTERNATIONAL,M5_V_MINUTES_TO_INTERNATIONAL,M5_V_REVENUE_TO_INTERNATIONAL,M5_GPRS_SESSION,M5_GPRS_REVENUE,M5_RECHARGES_NUMBER,M5_RECHARGES_VALUE,M5_GPRS_VOLUME,M5_OUT_DAYS,M5_INC_DAYS,M5_BUNDLE_REVENUE,M6_TTL_OUT_CALLS,M6_TTL_OUT_MINUTES,M6_TTL_OUT_REVENUE,M6_V_CALLS_TO_FIXED_LINES,M6_V_MINUTES_TO_FIXED_LINES,M6_V_REVENUE_TO_FIXED_LINES,M6_V_CALLS_TO_COMPETITION,M6_V_MINUTES_TO_COMPETITION,M6_V_REVENUE_TO_COMPETITION,M6_V_CALLS_TO_INTERNATIONAL,M6_V_MINUTES_TO_INTERNATIONAL,M6_V_REVENUE_TO_INTERNATIONAL,M6_GPRS_SESSION,M6_GPRS_REVENUE,M6_GPRS_VOLUME,M6_RECHARGES_NUMBER,M6_RECHARGES_VALUE,M6_OUT_DAYS,M6_INC_DAYS,M6_BUNDLE_REVENUE,TARIFF_PLAN,CONNECTION_DAY,SMARTPHONE_FLAG,INSERTED,M1_MINUTES_PER_CALL,M2_MINUTES_PER_CALL,M3_MINUTES_PER_CALL,M4_MINUTES_PER_CALL,M5_MINUTES_PER_CALL,M6_MINUTES_PER_CALL
0,6943089978,0,0.0,0.0000,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,19,0.0,600.000000,0,0.00,0,1,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.900000,0.0000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,30.0,0.0,731.959961,0.0,0.00,2.0,18.0,0.0,4.0,4.150000,3.1261,0.0,0.0,0.0,1.0,4.15,0.0,0.0,0.0,0.0,36.0,0.0,1000.000000,1.0,13.64,2.0,17.0,0.0,1.0,0.500000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,14.0,0.0,0.0,0.00,698.963867,1.0,0.0,0.0,2.0,5.483333,0.0000,0.0,0.000000,0.0,1.0,0.816667,0.00,0.0,0.0,0.0,17.0,0.0,501.036133,0.0,0.00,2.0,13.0,0.0,Cuba40,2014-08-28,Y,2022-04-30,NaN,NaN,0.633333,1.037500,0.500000,2.741667
1,6943938649,7,22.4,0.0000,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,86,0.0,1106.408776,2,21.82,2,26,10.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,29.483333,0.0000,0.0,0.0,0.0,3.0,26.250000,0.0,0.0,0.0,0.0,814.0,0.0,5263.889615,0.0,0.00,9.0,26.0,0.0,2.0,0.000000,1.5344,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,644.0,0.0,5566.920707,1.0,10.91,2.0,30.0,17.0,25.0,80.816667,0.0,1.0,0.250000,0.0,2.0,0.600000,0.0,0.0,0.0,0.0,390.0,0.0,0.0,0.00,3734.583810,13.0,31.0,2.5,13.0,23.233333,3.8288,0.0,0.000000,0.0,2.0,2.350000,

In [12]:
for column in df.columns.values:
    if (df[column].dtype == 'int64' or df[column].dtype == 'float64'):
        df[column] = df[column].fillna(0)
df.isnull().values.any()

False

# TENURE

In [13]:
# 2. Calculate tenure in months
tenure_in_months = ((df["INSERTED"] - df["CONNECTION_DAY"])/np.timedelta64(1, 'M'))
df["TENURE_MONTHS"]= tenure_in_months
df["TENURE_MONTHS"]= df["TENURE_MONTHS"].astype(int)

# Create categories according to tenure in years
customer_tenure = (tenure_in_months / 12)
customer_tenure_int = customer_tenure.astype('int64')

#labels = ["{0} - {1} yib".format(i, i + 1) for i in range(0, customer_tenure_int.max()+ 1, 1)]
#labels = ["0-9 yib", "10-19 yib", "20-29 yib", ">40 yib"]
labels = ["A", "B","C"]
# [..)
# df["CUSTOMER_TENURE"] = pd.cut(customer_tenure, bins=range(0, customer_tenure_int.max() + 2, 1), right=False, labels=labels)
df["CUSTOMER_TENURE"] = pd.cut(customer_tenure, bins=3, right=False, labels=labels)
df["CUSTOMER_TENURE"] = df["CUSTOMER_TENURE"].astype("category")

In [14]:
customer_tenure.max()

52.32687871756436

In [15]:
df["CUSTOMER_TENURE"].value_counts()

A    983680
B     42565
C        28
Name: CUSTOMER_TENURE, dtype: int64

# SET TERRIF TO CU

In [16]:
# 3. Replace tariff to be all CU
df["TARIFF_PLAN"] = df["TARIFF_PLAN"].replace({"Cuba40": "CU" , "Cuba": "CU"})

# ARPU

In [17]:
# 4. Calculate the ARPU (Average Revenue per Unit)
# find a method that detects automatically the months displayed on the dataset
# all datasets will be 6 months in duration
for month in range(1, 7):
    df[f"M{month}_ARPU"] = df[f"M{month}_TTL_OUT_REVENUE"] + df[f"M{month}_GPRS_REVENUE"] + df[f"M{month}_BUNDLE_REVENUE"]

In [18]:
df[["M1_TTL_OUT_REVENUE", "M1_GPRS_REVENUE", "M1_BUNDLE_REVENUE", "M1_ARPU"]].head(4)

,M1_TTL_OUT_REVENUE,M1_GPRS_REVENUE,M1_BUNDLE_REVENUE,M1_ARPU
0,0.0000,0.0,0.00,0.0000
1,0.0000,0.0,10.00,10.0000
2,0.0000,0.0,7.06,7.0600
3,0.5888,0.0,0.00,0.5888


# ROC

In [19]:
# 7a. Trend - Calculate rate of change of one month in comparison with the other months
month_check = 1
for column in df.columns.values:
    previous_months=[]
    if ((df[column].dtype == 'int64' or df[column].dtype == 'float64') and column!='MSISDN' and column!= 'TENURE_MONTHS'):
        for month in range(1,7):
            if month != month_check:
                previous_months.append(f"M{month}" + column[2:])
                df[f"M{month_check}_ROC"+ column[2:]] = ((df[f"M{month_check}" + column[2:]] - df[previous_months].mean(axis=1)) / df[previous_months].mean(axis=1)) * 100

In [20]:
# check the ROC
month_check = 1
previous_months = []
#check_roc=[]
for month in range(1,7):
    #if month != month_check:
    previous_months.append(f"M{month}_ARPU")
previous_months.append(f"M{month_check}_ROC_ARPU")

df[previous_months].head(5)

,M1_ARPU,M2_ARPU,M3_ARPU,M4_ARPU,M5_ARPU,M6_ARPU,M1_ROC_ARPU
0,0.0000,0.0,0.0000,3.1261,0.0,0.0000,-100.000000
1,10.0000,0.0,0.0000,18.5344,2.5,13.8288,43.417701
2,7.0600,0.0,0.0000,0.0000,0.0,0.0000,inf
3,0.5888,0.0,0.5888,9.5420,8.5,0.0000,-84.198209
4,0.0000,0.0,10.5888,0.0000,0.0,0.0000,-100.000000


# FILL NaNs

In [21]:
for column in df.columns.values:
    if (df[column].dtype == 'int64' or df[column].dtype == 'float64'):
        df[column] = df[column].fillna(0)
df.isnull().values.any()

False

# TAKE THE AVERAGE FOR HALF AND ALL PERIOD

In [22]:
# 7b. Create 3 month and 6 month averages for all usage columns
for column in df.columns.values:
    semester= []
    first_semester = []
    second_semester= []
    if ((df[column].dtype == 'int64' or df[column].dtype == 'float64') and column!='MSISDN' and column!= 'TENURE_MONTHS' and column[2:6]!="_ROC"):
        for month in range(1,4):
            first_semester.append(f"M{month}" + column[2:])
            semester.append(f"M{month}" + column[2:])
        df["M13_AVG"+ column[2:]] = df[first_semester].mean(axis=1)
        for month in range(4,7):
            second_semester.append(f"M{month}" + column[2:])
            semester.append(f"M{month}" + column[2:])
        df["M46_AVG"+ column[2:]] = df[second_semester].mean(axis=1)
        # calculate the average for all six months     
        df["M16_AVG"+ column[2:]] = df[semester].mean(axis=1)

In [23]:
# check the AVGs
ttl_out_calls = []
for month in range(1,7):
    ttl_out_calls.append(f"M{month}_TTL_OUT_CALLS")
ttl_out_calls.append("M16_AVG_TTL_OUT_CALLS")
df[ttl_out_calls].head(4)

,M1_TTL_OUT_CALLS,M2_TTL_OUT_CALLS,M3_TTL_OUT_CALLS,M4_TTL_OUT_CALLS,M5_TTL_OUT_CALLS,M6_TTL_OUT_CALLS,M16_AVG_TTL_OUT_CALLS
0,0,0.0,3.0,4.0,1.0,2.0,1.666667
1,7,0.0,12.0,2.0,25.0,13.0,9.833333
2,0,0.0,0.0,0.0,0.0,1.0,0.166667
3,2,0.0,14.0,2.0,45.0,29.0,15.333333


In [24]:
df.isnull().values.any()

False

In [25]:
df["M1_GPRS_VOLUME"].mean()

5888.446994652697

In [ ]:
df.shape()

# -----------------------------------------DEMOGRAPHICS -----------------------------------------

In [26]:
demographics["VF_COMBO_FLG"].value_counts()

N    1043551
Y      19554
Name: VF_COMBO_FLG, dtype: int64

In [27]:
demographics.head(4)

,POST_CODE,GENDER,AGE,MSISDN,VF_COMBO_FLG,ACTIVE_TOTAL_LINES
0,18453,M,47.0,6946234946,N,2.0
1,30002,O,74.0,6946816440,N,1.0
2,24100,M,46.0,6947221891,N,3.0
3,25100,M,30.0,6943454549,N,1.0


# Take care of NaNs

In [28]:
# 8. Fill NaNs in demographics dataset
demographics.isnull().values.any()
# COUNT how many NaN values we have per each column
for entry in list(demographics.columns.values):
    if demographics[entry].isnull().sum() > 0:
        print(entry, ": ", demographics[entry].isnull().sum())
        # I have to check the proportions (null / total input)
        print("The proportion of null values is: ", (demographics[entry].isnull().sum() / len(demographics[entry]) * 100, "%"))

True

POST_CODE :  314
The proportion of null values is:  (0.02953612296057304, '%')
GENDER :  6074
The proportion of null values is:  (0.571345257523951, '%')
AGE :  8486
The proportion of null values is:  (0.7982278326223656, '%')
ACTIVE_TOTAL_LINES :  419
The proportion of null values is:  (0.03941285197605128, '%')


In [29]:
# Check categorical values if they already have a None-type value
for column in demographics.columns.values:
    if (demographics[column].dtype == 'object' or demographics[column].dtype == 'datetime64[ns]'):
        demographics[column].value_counts()

DUMMY    120867
XXXXX     26552
85100     16953
45500     14686
30100      7954
          ...  
36076         1
10191         1
37009         1
10166         1
56501         1
Name: POST_CODE, Length: 1684, dtype: int64

O    645983
M    222978
F    188070
Name: GENDER, dtype: int64

N    1043551
Y      19554
Name: VF_COMBO_FLG, dtype: int64

In [30]:
demographics["POST_CODE"] = demographics["POST_CODE"].replace({"XXXXX": "DUMMY"})

In [31]:
for column in demographics.columns.values:
    if (demographics[column].dtype == 'int64' or demographics[column].dtype == 'float64'):
        # fill nulls with average of the column
        demographics[column] = demographics[column].fillna(demographics[column].mean())
    elif (demographics[column].dtype == 'object'):
        # in case the column is the gender I already have a value that stands for None
        if (demographics[column].name == "GENDER"):
            demographics[column] = demographics[column].fillna("O")
        if (demographics[column].name == "POST_CODE"):
            demographics[column] = demographics[column].fillna("DUMMY")
        demographics[column] = demographics[column].fillna("N/A")

In [32]:
demographics.isnull().values.any()
demographics.head(4)

False

,POST_CODE,GENDER,AGE,MSISDN,VF_COMBO_FLG,ACTIVE_TOTAL_LINES
0,18453,M,47.0,6946234946,N,2.0
1,30002,O,74.0,6946816440,N,1.0
2,24100,M,46.0,6947221891,N,3.0
3,25100,M,30.0,6943454549,N,1.0


# TAKE CARE OF STRANGE VALUES

In [33]:
demographics[demographics['AGE'] > 80].shape
demographics[demographics['AGE'] < 0].shape
demographics[demographics['AGE'] < 10].shape
demographics.shape

(16067, 6)

(7, 6)

(7, 6)

(1063105, 6)

In [34]:
# Replace "strange" ages with some threasholds
demographics['AGE'] = np.where(demographics['AGE'] > 80, 80, demographics['AGE'])
demographics['AGE'] = np.where(demographics['AGE'] < 10, 18, demographics['AGE'])
demographics['AGE'] = np.where(demographics['AGE'] < 0, 18, demographics['AGE'])

In [35]:
# Convert primary key of base usage datafrage to object
df["MSISDN"] = df["MSISDN"].astype('str')
df["MSISDN"].dtype

dtype('O')

# ---------------------------------------- CONSENT ------------------------------------

In [36]:
consent["MSISDN"]= consent["MSISDN"].astype(str)
consent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5115545 entries, 0 to 5115544
Data columns (total 1 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   MSISDN  object
dtypes: object(1)
memory usage: 39.0+ MB


In [37]:
consent.head(5)

,MSISDN
0,2100102312
1,2106532561
2,2610340453
3,2310228513
4,2103637517


# JOIN CUSTOMERS BASE USAGE WITH CONSENT

In [38]:
# Before join I have to check the shape of the dataframes I want to merge
df.shape
consent.shape

(1026273, 227)

(5115545, 1)

In [39]:
# 1o JOIN
customer_cnst = df.merge(consent, how='inner', on='MSISDN')
customer_cnst.head(4)

,MSISDN,M1_TTL_OUT_CALLS,M1_TTL_OUT_MINUTES,M1_TTL_OUT_REVENUE,M1_V_CALLS_TO_FIXED_LINES,M1_V_MINUTES_TO_FIXED_LINES,M1_V_REVENUE_TO_FIXED_LINES,M1_V_CALLS_TO_COMPETITION,M1_V_MINUTES_TO_COMPETITION,M1_V_REVENUE_TO_COMPETITION,M1_V_CALLS_TO_INTERNATIONAL,M1_V_MINUTES_TO_INTERNATIONAL,M1_V_REVENUE_TO_INTERNATIONAL,M1_GPRS_SESSION,M1_GPRS_REVENUE,M1_GPRS_VOLUME,M1_RECHARGES_NUMBER,M1_RECHARGES_VALUE,M1_OUT_DAYS,M1_INC_DAYS,M1_BUNDLE_REVENUE,M2_TTL_OUT_CALLS,M2_TTL_OUT_MINUTES,M2_TTL_OUT_REVENUE,M2_V_CALLS_TO_FIXED_LINES,M2_V_MINUTES_TO_FIXED_LINES,M2_V_REVENUE_TO_FIXED_LINES,M2_V_CALLS_TO_COMPETITION,M2_V_MINUTES_TO_COMPETITION,M2_V_REVENUE_TO_COMPETITION,M2_V_CALLS_TO_INTERNATIONAL,M2_V_MINUTES_TO_INTERNATIONAL,M2_V_REVENUE_TO_INTERNATIONAL,M2_GPRS_SESSION,M2_GPRS_REVENUE,M2_GPRS_VOLUME,M2_RECHARGES_NUMBER,M2_RECHARGES_VALUE,M2_OUT_DAYS,M2_INC_DAYS,M2_BUNDLE_REVENUE,M3_TTL_OUT_CALLS,M3_TTL_OUT_MINUTES,M3_TTL_OUT_REVENUE,M3_V_CALLS_TO_FIXED_LINES,M3_V_MINUTES_TO_FIXED_LINES,M3_V_REVENUE_TO_FIXED_LINES,M3_V_CALLS_TO_COMPETITION,M3_V_MINUTES_TO_COMPETITION,M3_V_REVENUE_TO_COMPETITION,M3_V_CALLS_TO_INTERNATIONAL,M3_V_MINUTES_TO_INTERNATIONAL,M3_V_REVENUE_TO_INTERNATIONAL,M3_GPRS_SESSION,M3_GPRS_REVENUE,M3_GPRS_VOLUME,M3_RECHARGES_NUMBER,M3_RECHARGES_VALUE,M3_OUT_DAYS,M3_INC_DAYS,M3_BUNDLE_REVENUE,M4_TTL_OUT_CALLS,M4_TTL_OUT_MINUTES,M4_TTL_OUT_REVENUE,M4_V_CALLS_TO_FIXED_LINES,M4_V_MINUTES_TO_FIXED_LINES,M4_V_REVENUE_TO_FIXED_LINES,M4_V_CALLS_TO_COMPETITION,M4_V_MINUTES_TO_COMPETITION,M4_V_REVENUE_TO_COMPETITION,M4_V_CALLS_TO_INTERNATIONAL,M4_V_MINUTES_TO_INTERNATIONAL,M4_V_REVENUE_TO_INTERNATIONAL,M4_GPRS_SESSION,M4_GPRS_REVENUE,M4_GPRS_VOLUME,M4_RECHARGES_NUMBER,M4_RECHARGES_VALUE,M4_OUT_DAYS,M4_INC_DAYS,M4_BUNDLE_REVENUE,M5_TTL_OUT_CALLS,M5_TTL_OUT_MINUTES,M5_TTL_OUT_REVENUE,M5_V_CALLS_TO_FIXED_LINES,M5_V_MINUTES_TO_FIXED_LINES,M5_V_REVENUE_TO_FIXED_LINES,M5_V_CALLS_TO_COMPETITION,M5_V_MINUTES_TO_COMPETITION,M5_V_REVENUE_TO_COMPETITION,M5_V_CALLS_TO_INTERNATIONAL,M5_V_MINUTES_TO_INTERNATIONAL,M5_V_REVENUE_TO_INTERNATIONAL,M5_GPRS_SESSION,M5_GPRS_REVENUE,M5_RECHARGES_NUMBER,M5_RECHARGES_VALUE,M5_GPRS_VOLUME,M5_OUT_DAYS,M5_INC_DAYS,M5_BUNDLE_REVENUE,M6_TTL_OUT_CALLS,M6_TTL_OUT_MINUTES,M6_TTL_OUT_REVENUE,M6_V_CALLS_TO_FIXED_LINES,M6_V_MINUTES_TO_FIXED_LINES,M6_V_REVENUE_TO_FIXED_LINES,M6_V_CALLS_TO_COMPETITION,M6_V_MINUTES_TO_COMPETITION,M6_V_REVENUE_TO_COMPETITION,M6_V_CALLS_TO_INTERNATIONAL,M6_V_MINUTES_TO_INTERNATIONAL,M6_V_REVENUE_TO_INTERNATIONAL,M6_GPRS_SESSION,M6_GPRS_REVENUE,M6_GPRS_VOLUME,M6_RECHARGES_NUMBER,M6_RECHARGES_VALUE,M6_OUT_DAYS,M6_INC_DAYS,M6_BUNDLE_REVENUE,TARIFF_PLAN,CONNECTION_DAY,SMARTPHONE_FLAG,INSERTED,M1_MINUTES_PER_CALL,M2_MINUTES_PER_CALL,M3_MINUTES_PER_CALL,M4_MINUTES_PER_CALL,M5_MINUTES_PER_CALL,M6_MINUTES_PER_CALL,TENURE_MONTHS,CUSTOMER_TENURE,M1_ARPU,M2_ARPU,M3_ARPU,M4_ARPU,M5_ARPU,M6_ARPU,M1_ROC_TTL_OUT_CALLS,M1_ROC_TTL_OUT_MINUTES,M1_ROC_TTL_OUT_REVENUE,M1_ROC_V_CALLS_TO_FIXED_LINES,M1_ROC_V_MINUTES_TO_FIXED_LINES,M1_ROC_V_REVENUE_TO_FIXED_LINES,M1_ROC_V_CALLS_TO_COMPETITION,M1_ROC_V_MINUTES_TO_COMPETITION,M1_ROC_V_REVENUE_TO_COMPETITION,M1_ROC_V_CALLS_TO_INTERNATIONAL,M1_ROC_V_MINUTES_TO_INTERNATIONAL,M1_ROC_V_REVENUE_TO_INTERNATIONAL,M1_ROC_GPRS_SESSION,M1_ROC_GPRS_REVENUE,M1_ROC_GPRS_VOLUME,M1_ROC_RECHARGES_NUMBER,M1_ROC_RECHARGES_VALUE,M1_ROC_OUT_DAYS,M1_ROC_INC_DAYS,M1_ROC_BUNDLE_REVENUE,M1_ROC_MINUTES_PER_CALL,M1_ROC_ARPU,M13_AVG_TTL_OUT_CALLS,M46_AVG_TTL_OUT_CALLS,M16_AVG_TTL_OUT_CALLS,M13_AVG_TTL_OUT_MINUTES,M46_AVG_TTL_OUT_MINUTES,M16_AVG_TTL_OUT_MINUTES,M13_AVG_TTL_OUT_REVENUE,M46_AVG_TTL_OUT_REVENUE,M16_AVG_TTL_OUT_REVENUE,M13_AVG_V_CALLS_TO_FIXED_LINES,M46_AVG_V_CALLS_TO_FIXED_LINES,M16_AVG_V_CALLS_TO_FIXED_LINES,M13_AVG_V_MINUTES_TO_FIXED_LINES,M46_AVG_V_MINUTES_TO_FIXED_LINES,M16_AVG_V_MINUTES_TO_FIXED_LINES,M13_AVG_V_REVENUE_TO_FIXED_LINES,M46_AVG_V_REVENUE_TO_FIXED_LINES,M16_AVG_V_REVENUE_TO_FIXED_LINES,M13_AVG_V_CALLS_TO_COMPETITION,M46_AVG_V_CALLS_TO_COMPETITION,M16_AVG_V_CALLS_TO

In [40]:
customer_cnst.shape

(907832, 227)

# JOIN DEMOGRAPHICS WITH CONSENT

In [41]:
demographics.shape

(1063105, 6)

In [42]:
# 2o JOIN
demographics["MSISDN"]=demographics["MSISDN"].astype(str)
demo_cnst = demographics.merge(consent, how="inner", on="MSISDN")
demo_cnst.head(4)

,POST_CODE,GENDER,AGE,MSISDN,VF_COMBO_FLG,ACTIVE_TOTAL_LINES
0,24100,M,46.0,6947221891,N,3.0
1,25100,M,30.0,6943454549,N,1.0
2,54352,M,23.0,6947555421,N,1.0
3,11144,O,38.0,6947993939,N,1.0


In [43]:
demo_cnst.shape

(939656, 6)

# JOIN STUDENTS WITH CONSENT

In [44]:
students["MSISDN"]=students["MSISDN"].astype(str)
students.shape

(402389, 2)

In [45]:
# 3o JOIN
stud_cnst = students.merge(consent, how="inner", on="MSISDN")
stud_cnst.head(4)

,MSISDN,STUDENTS_FLAG
0,6955535304,Y
1,6959409889,Y
2,6971771509,Y
3,6973882252,Y


In [46]:
stud_cnst.shape

(364495, 2)

In [47]:
stud_cnst[stud_cnst["MSISDN"].duplicated(keep=False)].shape

(0, 2)

In [48]:
stud_cnst.isnull().values.any()

False

# JOIN MAIN CUSTOMERS WITH STUDENTS

In [49]:
# 6o JOIN
customer_db_cnst = customer_cnst.merge(stud_cnst, how="left", on="MSISDN")

In [50]:
#customer_db_cnst
customer_db_cnst[customer_db_cnst["MSISDN"].duplicated(keep=False)].shape

(0, 228)

In [51]:
# Set N to customers that they aren't students
customer_db_cnst["STUDENTS_FLAG"]=customer_db_cnst["STUDENTS_FLAG"].fillna("N")
customer_db_cnst.head(4)

,MSISDN,M1_TTL_OUT_CALLS,M1_TTL_OUT_MINUTES,M1_TTL_OUT_REVENUE,M1_V_CALLS_TO_FIXED_LINES,M1_V_MINUTES_TO_FIXED_LINES,M1_V_REVENUE_TO_FIXED_LINES,M1_V_CALLS_TO_COMPETITION,M1_V_MINUTES_TO_COMPETITION,M1_V_REVENUE_TO_COMPETITION,M1_V_CALLS_TO_INTERNATIONAL,M1_V_MINUTES_TO_INTERNATIONAL,M1_V_REVENUE_TO_INTERNATIONAL,M1_GPRS_SESSION,M1_GPRS_REVENUE,M1_GPRS_VOLUME,M1_RECHARGES_NUMBER,M1_RECHARGES_VALUE,M1_OUT_DAYS,M1_INC_DAYS,M1_BUNDLE_REVENUE,M2_TTL_OUT_CALLS,M2_TTL_OUT_MINUTES,M2_TTL_OUT_REVENUE,M2_V_CALLS_TO_FIXED_LINES,M2_V_MINUTES_TO_FIXED_LINES,M2_V_REVENUE_TO_FIXED_LINES,M2_V_CALLS_TO_COMPETITION,M2_V_MINUTES_TO_COMPETITION,M2_V_REVENUE_TO_COMPETITION,M2_V_CALLS_TO_INTERNATIONAL,M2_V_MINUTES_TO_INTERNATIONAL,M2_V_REVENUE_TO_INTERNATIONAL,M2_GPRS_SESSION,M2_GPRS_REVENUE,M2_GPRS_VOLUME,M2_RECHARGES_NUMBER,M2_RECHARGES_VALUE,M2_OUT_DAYS,M2_INC_DAYS,M2_BUNDLE_REVENUE,M3_TTL_OUT_CALLS,M3_TTL_OUT_MINUTES,M3_TTL_OUT_REVENUE,M3_V_CALLS_TO_FIXED_LINES,M3_V_MINUTES_TO_FIXED_LINES,M3_V_REVENUE_TO_FIXED_LINES,M3_V_CALLS_TO_COMPETITION,M3_V_MINUTES_TO_COMPETITION,M3_V_REVENUE_TO_COMPETITION,M3_V_CALLS_TO_INTERNATIONAL,M3_V_MINUTES_TO_INTERNATIONAL,M3_V_REVENUE_TO_INTERNATIONAL,M3_GPRS_SESSION,M3_GPRS_REVENUE,M3_GPRS_VOLUME,M3_RECHARGES_NUMBER,M3_RECHARGES_VALUE,M3_OUT_DAYS,M3_INC_DAYS,M3_BUNDLE_REVENUE,M4_TTL_OUT_CALLS,M4_TTL_OUT_MINUTES,M4_TTL_OUT_REVENUE,M4_V_CALLS_TO_FIXED_LINES,M4_V_MINUTES_TO_FIXED_LINES,M4_V_REVENUE_TO_FIXED_LINES,M4_V_CALLS_TO_COMPETITION,M4_V_MINUTES_TO_COMPETITION,M4_V_REVENUE_TO_COMPETITION,M4_V_CALLS_TO_INTERNATIONAL,M4_V_MINUTES_TO_INTERNATIONAL,M4_V_REVENUE_TO_INTERNATIONAL,M4_GPRS_SESSION,M4_GPRS_REVENUE,M4_GPRS_VOLUME,M4_RECHARGES_NUMBER,M4_RECHARGES_VALUE,M4_OUT_DAYS,M4_INC_DAYS,M4_BUNDLE_REVENUE,M5_TTL_OUT_CALLS,M5_TTL_OUT_MINUTES,M5_TTL_OUT_REVENUE,M5_V_CALLS_TO_FIXED_LINES,M5_V_MINUTES_TO_FIXED_LINES,M5_V_REVENUE_TO_FIXED_LINES,M5_V_CALLS_TO_COMPETITION,M5_V_MINUTES_TO_COMPETITION,M5_V_REVENUE_TO_COMPETITION,M5_V_CALLS_TO_INTERNATIONAL,M5_V_MINUTES_TO_INTERNATIONAL,M5_V_REVENUE_TO_INTERNATIONAL,M5_GPRS_SESSION,M5_GPRS_REVENUE,M5_RECHARGES_NUMBER,M5_RECHARGES_VALUE,M5_GPRS_VOLUME,M5_OUT_DAYS,M5_INC_DAYS,M5_BUNDLE_REVENUE,M6_TTL_OUT_CALLS,M6_TTL_OUT_MINUTES,M6_TTL_OUT_REVENUE,M6_V_CALLS_TO_FIXED_LINES,M6_V_MINUTES_TO_FIXED_LINES,M6_V_REVENUE_TO_FIXED_LINES,M6_V_CALLS_TO_COMPETITION,M6_V_MINUTES_TO_COMPETITION,M6_V_REVENUE_TO_COMPETITION,M6_V_CALLS_TO_INTERNATIONAL,M6_V_MINUTES_TO_INTERNATIONAL,M6_V_REVENUE_TO_INTERNATIONAL,M6_GPRS_SESSION,M6_GPRS_REVENUE,M6_GPRS_VOLUME,M6_RECHARGES_NUMBER,M6_RECHARGES_VALUE,M6_OUT_DAYS,M6_INC_DAYS,M6_BUNDLE_REVENUE,TARIFF_PLAN,CONNECTION_DAY,SMARTPHONE_FLAG,INSERTED,M1_MINUTES_PER_CALL,M2_MINUTES_PER_CALL,M3_MINUTES_PER_CALL,M4_MINUTES_PER_CALL,M5_MINUTES_PER_CALL,M6_MINUTES_PER_CALL,TENURE_MONTHS,CUSTOMER_TENURE,M1_ARPU,M2_ARPU,M3_ARPU,M4_ARPU,M5_ARPU,M6_ARPU,M1_ROC_TTL_OUT_CALLS,M1_ROC_TTL_OUT_MINUTES,M1_ROC_TTL_OUT_REVENUE,M1_ROC_V_CALLS_TO_FIXED_LINES,M1_ROC_V_MINUTES_TO_FIXED_LINES,M1_ROC_V_REVENUE_TO_FIXED_LINES,M1_ROC_V_CALLS_TO_COMPETITION,M1_ROC_V_MINUTES_TO_COMPETITION,M1_ROC_V_REVENUE_TO_COMPETITION,M1_ROC_V_CALLS_TO_INTERNATIONAL,M1_ROC_V_MINUTES_TO_INTERNATIONAL,M1_ROC_V_REVENUE_TO_INTERNATIONAL,M1_ROC_GPRS_SESSION,M1_ROC_GPRS_REVENUE,M1_ROC_GPRS_VOLUME,M1_ROC_RECHARGES_NUMBER,M1_ROC_RECHARGES_VALUE,M1_ROC_OUT_DAYS,M1_ROC_INC_DAYS,M1_ROC_BUNDLE_REVENUE,M1_ROC_MINUTES_PER_CALL,M1_ROC_ARPU,M13_AVG_TTL_OUT_CALLS,M46_AVG_TTL_OUT_CALLS,M16_AVG_TTL_OUT_CALLS,M13_AVG_TTL_OUT_MINUTES,M46_AVG_TTL_OUT_MINUTES,M16_AVG_TTL_OUT_MINUTES,M13_AVG_TTL_OUT_REVENUE,M46_AVG_TTL_OUT_REVENUE,M16_AVG_TTL_OUT_REVENUE,M13_AVG_V_CALLS_TO_FIXED_LINES,M46_AVG_V_CALLS_TO_FIXED_LINES,M16_AVG_V_CALLS_TO_FIXED_LINES,M13_AVG_V_MINUTES_TO_FIXED_LINES,M46_AVG_V_MINUTES_TO_FIXED_LINES,M16_AVG_V_MINUTES_TO_FIXED_LINES,M13_AVG_V_REVENUE_TO_FIXED_LINES,M46_AVG_V_REVENUE_TO_FIXED_LINES,M16_AVG_V_REVENUE_TO_FIXED_LINES,M13_AVG_V_CALLS_TO_COMPETITION,M46_AVG_V_CALLS_TO_COMPETITION,M16_AVG_V_CALLS_TO

In [52]:
# check values after left join
customer_db_cnst["STUDENTS_FLAG"].value_counts()
customer_cnst.shape
stud_cnst.shape

N    556396
Y    351436
Name: STUDENTS_FLAG, dtype: int64

(907832, 227)

(364495, 2)

# JOIN ALL CUSTOMERS STUDENTS WITH DEMOGRAPHICS

In [53]:
# 7o JOIN
customer_demo_cnst = customer_db_cnst.merge(demo_cnst, how="left", on="MSISDN")
customer_demo_cnst.head(4)

,MSISDN,M1_TTL_OUT_CALLS,M1_TTL_OUT_MINUTES,M1_TTL_OUT_REVENUE,M1_V_CALLS_TO_FIXED_LINES,M1_V_MINUTES_TO_FIXED_LINES,M1_V_REVENUE_TO_FIXED_LINES,M1_V_CALLS_TO_COMPETITION,M1_V_MINUTES_TO_COMPETITION,M1_V_REVENUE_TO_COMPETITION,M1_V_CALLS_TO_INTERNATIONAL,M1_V_MINUTES_TO_INTERNATIONAL,M1_V_REVENUE_TO_INTERNATIONAL,M1_GPRS_SESSION,M1_GPRS_REVENUE,M1_GPRS_VOLUME,M1_RECHARGES_NUMBER,M1_RECHARGES_VALUE,M1_OUT_DAYS,M1_INC_DAYS,M1_BUNDLE_REVENUE,M2_TTL_OUT_CALLS,M2_TTL_OUT_MINUTES,M2_TTL_OUT_REVENUE,M2_V_CALLS_TO_FIXED_LINES,M2_V_MINUTES_TO_FIXED_LINES,M2_V_REVENUE_TO_FIXED_LINES,M2_V_CALLS_TO_COMPETITION,M2_V_MINUTES_TO_COMPETITION,M2_V_REVENUE_TO_COMPETITION,M2_V_CALLS_TO_INTERNATIONAL,M2_V_MINUTES_TO_INTERNATIONAL,M2_V_REVENUE_TO_INTERNATIONAL,M2_GPRS_SESSION,M2_GPRS_REVENUE,M2_GPRS_VOLUME,M2_RECHARGES_NUMBER,M2_RECHARGES_VALUE,M2_OUT_DAYS,M2_INC_DAYS,M2_BUNDLE_REVENUE,M3_TTL_OUT_CALLS,M3_TTL_OUT_MINUTES,M3_TTL_OUT_REVENUE,M3_V_CALLS_TO_FIXED_LINES,M3_V_MINUTES_TO_FIXED_LINES,M3_V_REVENUE_TO_FIXED_LINES,M3_V_CALLS_TO_COMPETITION,M3_V_MINUTES_TO_COMPETITION,M3_V_REVENUE_TO_COMPETITION,M3_V_CALLS_TO_INTERNATIONAL,M3_V_MINUTES_TO_INTERNATIONAL,M3_V_REVENUE_TO_INTERNATIONAL,M3_GPRS_SESSION,M3_GPRS_REVENUE,M3_GPRS_VOLUME,M3_RECHARGES_NUMBER,M3_RECHARGES_VALUE,M3_OUT_DAYS,M3_INC_DAYS,M3_BUNDLE_REVENUE,M4_TTL_OUT_CALLS,M4_TTL_OUT_MINUTES,M4_TTL_OUT_REVENUE,M4_V_CALLS_TO_FIXED_LINES,M4_V_MINUTES_TO_FIXED_LINES,M4_V_REVENUE_TO_FIXED_LINES,M4_V_CALLS_TO_COMPETITION,M4_V_MINUTES_TO_COMPETITION,M4_V_REVENUE_TO_COMPETITION,M4_V_CALLS_TO_INTERNATIONAL,M4_V_MINUTES_TO_INTERNATIONAL,M4_V_REVENUE_TO_INTERNATIONAL,M4_GPRS_SESSION,M4_GPRS_REVENUE,M4_GPRS_VOLUME,M4_RECHARGES_NUMBER,M4_RECHARGES_VALUE,M4_OUT_DAYS,M4_INC_DAYS,M4_BUNDLE_REVENUE,M5_TTL_OUT_CALLS,M5_TTL_OUT_MINUTES,M5_TTL_OUT_REVENUE,M5_V_CALLS_TO_FIXED_LINES,M5_V_MINUTES_TO_FIXED_LINES,M5_V_REVENUE_TO_FIXED_LINES,M5_V_CALLS_TO_COMPETITION,M5_V_MINUTES_TO_COMPETITION,M5_V_REVENUE_TO_COMPETITION,M5_V_CALLS_TO_INTERNATIONAL,M5_V_MINUTES_TO_INTERNATIONAL,M5_V_REVENUE_TO_INTERNATIONAL,M5_GPRS_SESSION,M5_GPRS_REVENUE,M5_RECHARGES_NUMBER,M5_RECHARGES_VALUE,M5_GPRS_VOLUME,M5_OUT_DAYS,M5_INC_DAYS,M5_BUNDLE_REVENUE,M6_TTL_OUT_CALLS,M6_TTL_OUT_MINUTES,M6_TTL_OUT_REVENUE,M6_V_CALLS_TO_FIXED_LINES,M6_V_MINUTES_TO_FIXED_LINES,M6_V_REVENUE_TO_FIXED_LINES,M6_V_CALLS_TO_COMPETITION,M6_V_MINUTES_TO_COMPETITION,M6_V_REVENUE_TO_COMPETITION,M6_V_CALLS_TO_INTERNATIONAL,M6_V_MINUTES_TO_INTERNATIONAL,M6_V_REVENUE_TO_INTERNATIONAL,M6_GPRS_SESSION,M6_GPRS_REVENUE,M6_GPRS_VOLUME,M6_RECHARGES_NUMBER,M6_RECHARGES_VALUE,M6_OUT_DAYS,M6_INC_DAYS,M6_BUNDLE_REVENUE,TARIFF_PLAN,CONNECTION_DAY,SMARTPHONE_FLAG,INSERTED,M1_MINUTES_PER_CALL,M2_MINUTES_PER_CALL,M3_MINUTES_PER_CALL,M4_MINUTES_PER_CALL,M5_MINUTES_PER_CALL,M6_MINUTES_PER_CALL,TENURE_MONTHS,CUSTOMER_TENURE,M1_ARPU,M2_ARPU,M3_ARPU,M4_ARPU,M5_ARPU,M6_ARPU,M1_ROC_TTL_OUT_CALLS,M1_ROC_TTL_OUT_MINUTES,M1_ROC_TTL_OUT_REVENUE,M1_ROC_V_CALLS_TO_FIXED_LINES,M1_ROC_V_MINUTES_TO_FIXED_LINES,M1_ROC_V_REVENUE_TO_FIXED_LINES,M1_ROC_V_CALLS_TO_COMPETITION,M1_ROC_V_MINUTES_TO_COMPETITION,M1_ROC_V_REVENUE_TO_COMPETITION,M1_ROC_V_CALLS_TO_INTERNATIONAL,M1_ROC_V_MINUTES_TO_INTERNATIONAL,M1_ROC_V_REVENUE_TO_INTERNATIONAL,M1_ROC_GPRS_SESSION,M1_ROC_GPRS_REVENUE,M1_ROC_GPRS_VOLUME,M1_ROC_RECHARGES_NUMBER,M1_ROC_RECHARGES_VALUE,M1_ROC_OUT_DAYS,M1_ROC_INC_DAYS,M1_ROC_BUNDLE_REVENUE,M1_ROC_MINUTES_PER_CALL,M1_ROC_ARPU,M13_AVG_TTL_OUT_CALLS,M46_AVG_TTL_OUT_CALLS,M16_AVG_TTL_OUT_CALLS,M13_AVG_TTL_OUT_MINUTES,M46_AVG_TTL_OUT_MINUTES,M16_AVG_TTL_OUT_MINUTES,M13_AVG_TTL_OUT_REVENUE,M46_AVG_TTL_OUT_REVENUE,M16_AVG_TTL_OUT_REVENUE,M13_AVG_V_CALLS_TO_FIXED_LINES,M46_AVG_V_CALLS_TO_FIXED_LINES,M16_AVG_V_CALLS_TO_FIXED_LINES,M13_AVG_V_MINUTES_TO_FIXED_LINES,M46_AVG_V_MINUTES_TO_FIXED_LINES,M16_AVG_V_MINUTES_TO_FIXED_LINES,M13_AVG_V_REVENUE_TO_FIXED_LINES,M46_AVG_V_REVENUE_TO_FIXED_LINES,M16_AVG_V_REVENUE_TO_FIXED_LINES,M13_AVG_V_CALLS_TO_COMPETITION,M46_AVG_V_CALLS_TO_COMPETITION,M16_AVG_V_CALLS_TO

In [54]:
customer_demo_cnst.isnull().values.any()

True

In [55]:
# Fill occure NaNs values
for column in customer_demo_cnst.columns.values:
    if (customer_demo_cnst[column].dtype == 'int64' or customer_demo_cnst[column].dtype == 'float64'):
        # chech strange age values
        customer_demo_cnst[column] = customer_demo_cnst[column].fillna(customer_demo_cnst[column].mean())
    elif (customer_demo_cnst[column].dtype == 'object' or customer_demo_cnst[column].dtype == 'str'):
        # in case the column is the gender I already have a value that stands for None
        if (customer_demo_cnst[column].name == "GENDER"):
            customer_demo_cnst[column] = customer_demo_cnst[column].fillna("O")
        customer_demo_cnst[column] = customer_demo_cnst[column].fillna("N/A")

In [56]:
customer_demo_cnst.isnull().values.any()

False

In [57]:
customer_demo_cnst["GENDER"].value_counts()
customer_db_cnst.shape
demo_cnst.shape

O    557839
M    187241
F    162752
Name: GENDER, dtype: int64

(907832, 228)

(939656, 6)

# ----------------------------------------- POST CODE ------------------------------------------

In [58]:
customer_demo_cnst.head()
customer_demo_cnst["POST_CODE"]=customer_demo_cnst["POST_CODE"].astype(str)

,MSISDN,M1_TTL_OUT_CALLS,M1_TTL_OUT_MINUTES,M1_TTL_OUT_REVENUE,M1_V_CALLS_TO_FIXED_LINES,M1_V_MINUTES_TO_FIXED_LINES,M1_V_REVENUE_TO_FIXED_LINES,M1_V_CALLS_TO_COMPETITION,M1_V_MINUTES_TO_COMPETITION,M1_V_REVENUE_TO_COMPETITION,M1_V_CALLS_TO_INTERNATIONAL,M1_V_MINUTES_TO_INTERNATIONAL,M1_V_REVENUE_TO_INTERNATIONAL,M1_GPRS_SESSION,M1_GPRS_REVENUE,M1_GPRS_VOLUME,M1_RECHARGES_NUMBER,M1_RECHARGES_VALUE,M1_OUT_DAYS,M1_INC_DAYS,M1_BUNDLE_REVENUE,M2_TTL_OUT_CALLS,M2_TTL_OUT_MINUTES,M2_TTL_OUT_REVENUE,M2_V_CALLS_TO_FIXED_LINES,M2_V_MINUTES_TO_FIXED_LINES,M2_V_REVENUE_TO_FIXED_LINES,M2_V_CALLS_TO_COMPETITION,M2_V_MINUTES_TO_COMPETITION,M2_V_REVENUE_TO_COMPETITION,M2_V_CALLS_TO_INTERNATIONAL,M2_V_MINUTES_TO_INTERNATIONAL,M2_V_REVENUE_TO_INTERNATIONAL,M2_GPRS_SESSION,M2_GPRS_REVENUE,M2_GPRS_VOLUME,M2_RECHARGES_NUMBER,M2_RECHARGES_VALUE,M2_OUT_DAYS,M2_INC_DAYS,M2_BUNDLE_REVENUE,M3_TTL_OUT_CALLS,M3_TTL_OUT_MINUTES,M3_TTL_OUT_REVENUE,M3_V_CALLS_TO_FIXED_LINES,M3_V_MINUTES_TO_FIXED_LINES,M3_V_REVENUE_TO_FIXED_LINES,M3_V_CALLS_TO_COMPETITION,M3_V_MINUTES_TO_COMPETITION,M3_V_REVENUE_TO_COMPETITION,M3_V_CALLS_TO_INTERNATIONAL,M3_V_MINUTES_TO_INTERNATIONAL,M3_V_REVENUE_TO_INTERNATIONAL,M3_GPRS_SESSION,M3_GPRS_REVENUE,M3_GPRS_VOLUME,M3_RECHARGES_NUMBER,M3_RECHARGES_VALUE,M3_OUT_DAYS,M3_INC_DAYS,M3_BUNDLE_REVENUE,M4_TTL_OUT_CALLS,M4_TTL_OUT_MINUTES,M4_TTL_OUT_REVENUE,M4_V_CALLS_TO_FIXED_LINES,M4_V_MINUTES_TO_FIXED_LINES,M4_V_REVENUE_TO_FIXED_LINES,M4_V_CALLS_TO_COMPETITION,M4_V_MINUTES_TO_COMPETITION,M4_V_REVENUE_TO_COMPETITION,M4_V_CALLS_TO_INTERNATIONAL,M4_V_MINUTES_TO_INTERNATIONAL,M4_V_REVENUE_TO_INTERNATIONAL,M4_GPRS_SESSION,M4_GPRS_REVENUE,M4_GPRS_VOLUME,M4_RECHARGES_NUMBER,M4_RECHARGES_VALUE,M4_OUT_DAYS,M4_INC_DAYS,M4_BUNDLE_REVENUE,M5_TTL_OUT_CALLS,M5_TTL_OUT_MINUTES,M5_TTL_OUT_REVENUE,M5_V_CALLS_TO_FIXED_LINES,M5_V_MINUTES_TO_FIXED_LINES,M5_V_REVENUE_TO_FIXED_LINES,M5_V_CALLS_TO_COMPETITION,M5_V_MINUTES_TO_COMPETITION,M5_V_REVENUE_TO_COMPETITION,M5_V_CALLS_TO_INTERNATIONAL,M5_V_MINUTES_TO_INTERNATIONAL,M5_V_REVENUE_TO_INTERNATIONAL,M5_GPRS_SESSION,M5_GPRS_REVENUE,M5_RECHARGES_NUMBER,M5_RECHARGES_VALUE,M5_GPRS_VOLUME,M5_OUT_DAYS,M5_INC_DAYS,M5_BUNDLE_REVENUE,M6_TTL_OUT_CALLS,M6_TTL_OUT_MINUTES,M6_TTL_OUT_REVENUE,M6_V_CALLS_TO_FIXED_LINES,M6_V_MINUTES_TO_FIXED_LINES,M6_V_REVENUE_TO_FIXED_LINES,M6_V_CALLS_TO_COMPETITION,M6_V_MINUTES_TO_COMPETITION,M6_V_REVENUE_TO_COMPETITION,M6_V_CALLS_TO_INTERNATIONAL,M6_V_MINUTES_TO_INTERNATIONAL,M6_V_REVENUE_TO_INTERNATIONAL,M6_GPRS_SESSION,M6_GPRS_REVENUE,M6_GPRS_VOLUME,M6_RECHARGES_NUMBER,M6_RECHARGES_VALUE,M6_OUT_DAYS,M6_INC_DAYS,M6_BUNDLE_REVENUE,TARIFF_PLAN,CONNECTION_DAY,SMARTPHONE_FLAG,INSERTED,M1_MINUTES_PER_CALL,M2_MINUTES_PER_CALL,M3_MINUTES_PER_CALL,M4_MINUTES_PER_CALL,M5_MINUTES_PER_CALL,M6_MINUTES_PER_CALL,TENURE_MONTHS,CUSTOMER_TENURE,M1_ARPU,M2_ARPU,M3_ARPU,M4_ARPU,M5_ARPU,M6_ARPU,M1_ROC_TTL_OUT_CALLS,M1_ROC_TTL_OUT_MINUTES,M1_ROC_TTL_OUT_REVENUE,M1_ROC_V_CALLS_TO_FIXED_LINES,M1_ROC_V_MINUTES_TO_FIXED_LINES,M1_ROC_V_REVENUE_TO_FIXED_LINES,M1_ROC_V_CALLS_TO_COMPETITION,M1_ROC_V_MINUTES_TO_COMPETITION,M1_ROC_V_REVENUE_TO_COMPETITION,M1_ROC_V_CALLS_TO_INTERNATIONAL,M1_ROC_V_MINUTES_TO_INTERNATIONAL,M1_ROC_V_REVENUE_TO_INTERNATIONAL,M1_ROC_GPRS_SESSION,M1_ROC_GPRS_REVENUE,M1_ROC_GPRS_VOLUME,M1_ROC_RECHARGES_NUMBER,M1_ROC_RECHARGES_VALUE,M1_ROC_OUT_DAYS,M1_ROC_INC_DAYS,M1_ROC_BUNDLE_REVENUE,M1_ROC_MINUTES_PER_CALL,M1_ROC_ARPU,M13_AVG_TTL_OUT_CALLS,M46_AVG_TTL_OUT_CALLS,M16_AVG_TTL_OUT_CALLS,M13_AVG_TTL_OUT_MINUTES,M46_AVG_TTL_OUT_MINUTES,M16_AVG_TTL_OUT_MINUTES,M13_AVG_TTL_OUT_REVENUE,M46_AVG_TTL_OUT_REVENUE,M16_AVG_TTL_OUT_REVENUE,M13_AVG_V_CALLS_TO_FIXED_LINES,M46_AVG_V_CALLS_TO_FIXED_LINES,M16_AVG_V_CALLS_TO_FIXED_LINES,M13_AVG_V_MINUTES_TO_FIXED_LINES,M46_AVG_V_MINUTES_TO_FIXED_LINES,M16_AVG_V_MINUTES_TO_FIXED_LINES,M13_AVG_V_REVENUE_TO_FIXED_LINES,M46_AVG_V_REVENUE_TO_FIXED_LINES,M16_AVG_V_REVENUE_TO_FIXED_LINES,M13_AVG_V_CALLS_TO_COMPETITION,M46_AVG_V_CALLS_TO_COMPETITION,M16_AVG_V_CALLS_TO

In [59]:
post_code["POST_CODE"]= post_code["POST_CODE"].astype(str)

# I want to drop the columns with users that are 60+ years old
for column in post_code.columns.values:
    if "60+" in column:
        post_code = post_code.drop(column, axis="columns")

# Take care of NaNs

In [60]:
post_code.isnull().values.any()

False

In [61]:
# Check categorical values if they already have a None-type value
for column in post_code.columns.values:
    if (post_code[column].dtype == 'object' or post_code[column].dtype == 'str' or post_code[column].dtype == 'datetime64[ns]'):
        post_code[column].value_counts()

Kentrikos Tomeas Athinon      113
Thessaloniki                   89
Voreios Tomeas Athinon         51
Anatoliki Attiki               42
Peiraias                       32
Notios Tomeas Athinon          30
Aitolia kai Akarnania          28
Achaia                         27
Dytikos Tomeas Athinon         22
Evoia                          20
Ioannina                       20
Chania                         20
Ilia                           18
Messinia                       18
Fthiotida                      17
Rodos                          17
Iraklio                        16
Larisa                         16
Magnisia                       15
Kavala                         15
Arkadia                        15
Serres                         15
Kozani                         15
Dytiki Attiki                  14
Lesvos                         14
Evros                          14
Korinthia                      13
Chalkidiki                     12
Voiotia                        11
Rethymno      

82103    1
84302    1
56535    1
45333    1
56121    1
        ..
70004    1
63077    1
82150    1
68133    1
54625    1
Name: POST_CODE, Length: 993, dtype: int64

Athinaíon                                    90
Thessaloníkis                                32
Peiraiós                                     17
Patréon                                      17
Lésvou                                       14
Ródou                                        13
Irakleíou                                    13
Ioannitón                                    11
Agriníou                                     11
Pávlou Melá                                  10
Chaníon                                      10
Vólou                                        10
Acharnón                                      9
Larisaíon                                     9
Kaválas                                       9
Kérkyras                                      8
Dionýsou                                      7
Serrón                                        7
Peristeríou                                   7
Kifisiás                                      6
Kallithéas                              

In [62]:
# Set one common None value
post_code["POST_CODE"] = post_code["POST_CODE"].replace({"XXXXX": "DUMMY"})

In [63]:
post_code.columns.values.dtype

dtype('O')

In [64]:
post_code.head(4)

,Periferiaki_enotita,POST_CODE,Name,Population,Households,Average_Household_Size,Male_Population,Female_Population,Population_aged_0-14,Male_Population_aged_0-14,Female_Population_aged_0-14,Population_aged_15-29,Male_Population_aged_15-29,Female_Population_aged_15-29,Population_aged_30-44,Male_Population_aged_30-44,Female_Population_aged_30-44,Population_aged_45-59,Male_Population_aged_45-59,Female_Population_aged_45-59,Purchasing_Power:_million_Euro,Purchasing_Power:_per_mill_of_country,Purchasing_Power:_Euro_per_capita,Purchasing_Power:_index_(country_eq.100)
0,Kentrikos Tomeas Athinon,10431,Athinaíon,1089,489,2.23,520,569,163,77,86,146,75,71,198,97,101,250,118,132,15.68,0.15,14397.50,139.40
1,Kentrikos Tomeas Athinon,10432,Athinaíon,1166,525,2.22,561,605,172,83,89,149,75,74,216,104,112,245,123,122,16.73,0.16,14350.82,138.95
2,Kentrikos Tomeas Athinon,10433,Athinaíon,4087,1832,2.23,1925,2162,783,391,392,653,325,328,884,409,475,900,412,488,65.35,0.61,15990.07,154.82
3,Kentrikos Tomeas Athinon,10434,Athinaíon,6149,2755,2.23,2910,3239,1156,575,581,949,478,471,1285,607,678,1344,617,727,91.04,0.85,14805.50,143.35


# PURCHASING POWER PERCENTAGE

In [65]:
for column in post_code.columns.values:
    if ("Purchasing_Power" in column):
        post_code[column + "_percentage"] = (post_code[column] / post_code[column].mean()) * 100
#customer_demo_post_cnst
#customer_demo_post_cnst["Purchasing_Power:_Euro_per_capita"].max()
# NA DIAIRESW ME TO MAX/AVG

In [66]:
post_code.head()

,Periferiaki_enotita,POST_CODE,Name,Population,Households,Average_Household_Size,Male_Population,Female_Population,Population_aged_0-14,Male_Population_aged_0-14,Female_Population_aged_0-14,Population_aged_15-29,Male_Population_aged_15-29,Female_Population_aged_15-29,Population_aged_30-44,Male_Population_aged_30-44,Female_Population_aged_30-44,Population_aged_45-59,Male_Population_aged_45-59,Female_Population_aged_45-59,Purchasing_Power:_million_Euro,Purchasing_Power:_per_mill_of_country,Purchasing_Power:_Euro_per_capita,Purchasing_Power:_index_(country_eq.100),Purchasing_Power:_million_Euro_percentage,Purchasing_Power:_per_mill_of_country_percentage,Purchasing_Power:_Euro_per_capita_percentage,Purchasing_Power:_index_(country_eq.100)_percentage
0,Kentrikos Tomeas Athinon,10431,Athinaíon,1089,489,2.23,520,569,163,77,86,146,75,71,198,97,101,250,118,132,15.68,0.15,14397.50,139.40,14.497996,14.896341,139.779233,139.775310
1,Kentrikos Tomeas Athinon,10432,Athinaíon,1166,525,2.22,561,605,172,83,89,149,75,74,216,104,112,245,123,122,16.73,0.16,14350.82,138.95,15.468844,15.889430,139.326036,139.324099
2,Kentrikos Tomeas Athinon,10433,Athinaíon,4087,1832,2.23,1925,2162,783,391,392,653,325,328,884,409,475,900,412,488,65.35,0.61,15990.07,154.82,60.423726,60.578452,155.240821,155.236826
3,Kentrikos Tomeas Athinon,10434,Athinaíon,6149,2755,2.23,2910,3239,1156,575,581,949,478,471,1285,607,678,1344,617,727,91.04,0.85,14805.50,143.35,84.177139,84.412597,143.740332,143.735945
4,Kentrikos Tomeas Athinon,10435,Athinaíon,4627,2075,2.23,2181,2446,890,443,447,738,367,371,997,462,535,1016,466,550,54.19,0.50,11710.98,113.39,50.104999,49.654469,113.696947,113.695283


# JOIN ALL PREVIOUS DATASETS WITH POST CODE

In [67]:
# 8o JOIN
customer_demo_post_cnst = customer_demo_cnst.merge(post_code, how="left", on="POST_CODE")
customer_demo_post_cnst.head(4)

,MSISDN,M1_TTL_OUT_CALLS,M1_TTL_OUT_MINUTES,M1_TTL_OUT_REVENUE,M1_V_CALLS_TO_FIXED_LINES,M1_V_MINUTES_TO_FIXED_LINES,M1_V_REVENUE_TO_FIXED_LINES,M1_V_CALLS_TO_COMPETITION,M1_V_MINUTES_TO_COMPETITION,M1_V_REVENUE_TO_COMPETITION,M1_V_CALLS_TO_INTERNATIONAL,M1_V_MINUTES_TO_INTERNATIONAL,M1_V_REVENUE_TO_INTERNATIONAL,M1_GPRS_SESSION,M1_GPRS_REVENUE,M1_GPRS_VOLUME,M1_RECHARGES_NUMBER,M1_RECHARGES_VALUE,M1_OUT_DAYS,M1_INC_DAYS,M1_BUNDLE_REVENUE,M2_TTL_OUT_CALLS,M2_TTL_OUT_MINUTES,M2_TTL_OUT_REVENUE,M2_V_CALLS_TO_FIXED_LINES,M2_V_MINUTES_TO_FIXED_LINES,M2_V_REVENUE_TO_FIXED_LINES,M2_V_CALLS_TO_COMPETITION,M2_V_MINUTES_TO_COMPETITION,M2_V_REVENUE_TO_COMPETITION,M2_V_CALLS_TO_INTERNATIONAL,M2_V_MINUTES_TO_INTERNATIONAL,M2_V_REVENUE_TO_INTERNATIONAL,M2_GPRS_SESSION,M2_GPRS_REVENUE,M2_GPRS_VOLUME,M2_RECHARGES_NUMBER,M2_RECHARGES_VALUE,M2_OUT_DAYS,M2_INC_DAYS,M2_BUNDLE_REVENUE,M3_TTL_OUT_CALLS,M3_TTL_OUT_MINUTES,M3_TTL_OUT_REVENUE,M3_V_CALLS_TO_FIXED_LINES,M3_V_MINUTES_TO_FIXED_LINES,M3_V_REVENUE_TO_FIXED_LINES,M3_V_CALLS_TO_COMPETITION,M3_V_MINUTES_TO_COMPETITION,M3_V_REVENUE_TO_COMPETITION,M3_V_CALLS_TO_INTERNATIONAL,M3_V_MINUTES_TO_INTERNATIONAL,M3_V_REVENUE_TO_INTERNATIONAL,M3_GPRS_SESSION,M3_GPRS_REVENUE,M3_GPRS_VOLUME,M3_RECHARGES_NUMBER,M3_RECHARGES_VALUE,M3_OUT_DAYS,M3_INC_DAYS,M3_BUNDLE_REVENUE,M4_TTL_OUT_CALLS,M4_TTL_OUT_MINUTES,M4_TTL_OUT_REVENUE,M4_V_CALLS_TO_FIXED_LINES,M4_V_MINUTES_TO_FIXED_LINES,M4_V_REVENUE_TO_FIXED_LINES,M4_V_CALLS_TO_COMPETITION,M4_V_MINUTES_TO_COMPETITION,M4_V_REVENUE_TO_COMPETITION,M4_V_CALLS_TO_INTERNATIONAL,M4_V_MINUTES_TO_INTERNATIONAL,M4_V_REVENUE_TO_INTERNATIONAL,M4_GPRS_SESSION,M4_GPRS_REVENUE,M4_GPRS_VOLUME,M4_RECHARGES_NUMBER,M4_RECHARGES_VALUE,M4_OUT_DAYS,M4_INC_DAYS,M4_BUNDLE_REVENUE,M5_TTL_OUT_CALLS,M5_TTL_OUT_MINUTES,M5_TTL_OUT_REVENUE,M5_V_CALLS_TO_FIXED_LINES,M5_V_MINUTES_TO_FIXED_LINES,M5_V_REVENUE_TO_FIXED_LINES,M5_V_CALLS_TO_COMPETITION,M5_V_MINUTES_TO_COMPETITION,M5_V_REVENUE_TO_COMPETITION,M5_V_CALLS_TO_INTERNATIONAL,M5_V_MINUTES_TO_INTERNATIONAL,M5_V_REVENUE_TO_INTERNATIONAL,M5_GPRS_SESSION,M5_GPRS_REVENUE,M5_RECHARGES_NUMBER,M5_RECHARGES_VALUE,M5_GPRS_VOLUME,M5_OUT_DAYS,M5_INC_DAYS,M5_BUNDLE_REVENUE,M6_TTL_OUT_CALLS,M6_TTL_OUT_MINUTES,M6_TTL_OUT_REVENUE,M6_V_CALLS_TO_FIXED_LINES,M6_V_MINUTES_TO_FIXED_LINES,M6_V_REVENUE_TO_FIXED_LINES,M6_V_CALLS_TO_COMPETITION,M6_V_MINUTES_TO_COMPETITION,M6_V_REVENUE_TO_COMPETITION,M6_V_CALLS_TO_INTERNATIONAL,M6_V_MINUTES_TO_INTERNATIONAL,M6_V_REVENUE_TO_INTERNATIONAL,M6_GPRS_SESSION,M6_GPRS_REVENUE,M6_GPRS_VOLUME,M6_RECHARGES_NUMBER,M6_RECHARGES_VALUE,M6_OUT_DAYS,M6_INC_DAYS,M6_BUNDLE_REVENUE,TARIFF_PLAN,CONNECTION_DAY,SMARTPHONE_FLAG,INSERTED,M1_MINUTES_PER_CALL,M2_MINUTES_PER_CALL,M3_MINUTES_PER_CALL,M4_MINUTES_PER_CALL,M5_MINUTES_PER_CALL,M6_MINUTES_PER_CALL,TENURE_MONTHS,CUSTOMER_TENURE,M1_ARPU,M2_ARPU,M3_ARPU,M4_ARPU,M5_ARPU,M6_ARPU,M1_ROC_TTL_OUT_CALLS,M1_ROC_TTL_OUT_MINUTES,M1_ROC_TTL_OUT_REVENUE,M1_ROC_V_CALLS_TO_FIXED_LINES,M1_ROC_V_MINUTES_TO_FIXED_LINES,M1_ROC_V_REVENUE_TO_FIXED_LINES,M1_ROC_V_CALLS_TO_COMPETITION,M1_ROC_V_MINUTES_TO_COMPETITION,M1_ROC_V_REVENUE_TO_COMPETITION,M1_ROC_V_CALLS_TO_INTERNATIONAL,M1_ROC_V_MINUTES_TO_INTERNATIONAL,M1_ROC_V_REVENUE_TO_INTERNATIONAL,M1_ROC_GPRS_SESSION,M1_ROC_GPRS_REVENUE,M1_ROC_GPRS_VOLUME,M1_ROC_RECHARGES_NUMBER,M1_ROC_RECHARGES_VALUE,M1_ROC_OUT_DAYS,M1_ROC_INC_DAYS,M1_ROC_BUNDLE_REVENUE,M1_ROC_MINUTES_PER_CALL,M1_ROC_ARPU,M13_AVG_TTL_OUT_CALLS,M46_AVG_TTL_OUT_CALLS,M16_AVG_TTL_OUT_CALLS,M13_AVG_TTL_OUT_MINUTES,M46_AVG_TTL_OUT_MINUTES,M16_AVG_TTL_OUT_MINUTES,M13_AVG_TTL_OUT_REVENUE,M46_AVG_TTL_OUT_REVENUE,M16_AVG_TTL_OUT_REVENUE,M13_AVG_V_CALLS_TO_FIXED_LINES,M46_AVG_V_CALLS_TO_FIXED_LINES,M16_AVG_V_CALLS_TO_FIXED_LINES,M13_AVG_V_MINUTES_TO_FIXED_LINES,M46_AVG_V_MINUTES_TO_FIXED_LINES,M16_AVG_V_MINUTES_TO_FIXED_LINES,M13_AVG_V_REVENUE_TO_FIXED_LINES,M46_AVG_V_REVENUE_TO_FIXED_LINES,M16_AVG_V_REVENUE_TO_FIXED_LINES,M13_AVG_V_CALLS_TO_COMPETITION,M46_AVG_V_CALLS_TO_COMPETITION,M16_AVG_V_CALLS_TO

In [68]:
customer_demo_post_cnst.isnull().values.any()

True

In [69]:
# Fill occure NaNs values
for column in customer_demo_post_cnst.columns.values:
    if (customer_demo_post_cnst[column].dtype == 'int64' or customer_demo_post_cnst[column].dtype == 'float64'):
        customer_demo_post_cnst[column] = customer_demo_post_cnst[column].fillna(customer_demo_post_cnst[column].mean())
    elif (customer_demo_post_cnst[column].dtype == 'object' or customer_demo_post_cnst[column].dtype == 'str'):
        customer_demo_post_cnst[column] = customer_demo_post_cnst[column].fillna("N/A")

In [70]:
customer_demo_post_cnst.isnull().values.any()

False

In [71]:
#customer_demo_post_cnst["POST_CODE"].value_counts()
customer_demo_cnst.shape
post_code.shape

(907832, 233)

(993, 28)

In [72]:
###########################################################

# Final check for NaNs

In [73]:
# after all transformations
customer_demo_post_cnst = customer_demo_post_cnst.replace([np.inf, -np.inf, np.nan], 0.0)

In [74]:
customer_demo_post_cnst.isnull().values.any()

False

# Save

In [75]:
customer_demo_post_cnst.head()
customer_demo_post_cnst.to_csv(out_dir)

,MSISDN,M1_TTL_OUT_CALLS,M1_TTL_OUT_MINUTES,M1_TTL_OUT_REVENUE,M1_V_CALLS_TO_FIXED_LINES,M1_V_MINUTES_TO_FIXED_LINES,M1_V_REVENUE_TO_FIXED_LINES,M1_V_CALLS_TO_COMPETITION,M1_V_MINUTES_TO_COMPETITION,M1_V_REVENUE_TO_COMPETITION,M1_V_CALLS_TO_INTERNATIONAL,M1_V_MINUTES_TO_INTERNATIONAL,M1_V_REVENUE_TO_INTERNATIONAL,M1_GPRS_SESSION,M1_GPRS_REVENUE,M1_GPRS_VOLUME,M1_RECHARGES_NUMBER,M1_RECHARGES_VALUE,M1_OUT_DAYS,M1_INC_DAYS,M1_BUNDLE_REVENUE,M2_TTL_OUT_CALLS,M2_TTL_OUT_MINUTES,M2_TTL_OUT_REVENUE,M2_V_CALLS_TO_FIXED_LINES,M2_V_MINUTES_TO_FIXED_LINES,M2_V_REVENUE_TO_FIXED_LINES,M2_V_CALLS_TO_COMPETITION,M2_V_MINUTES_TO_COMPETITION,M2_V_REVENUE_TO_COMPETITION,M2_V_CALLS_TO_INTERNATIONAL,M2_V_MINUTES_TO_INTERNATIONAL,M2_V_REVENUE_TO_INTERNATIONAL,M2_GPRS_SESSION,M2_GPRS_REVENUE,M2_GPRS_VOLUME,M2_RECHARGES_NUMBER,M2_RECHARGES_VALUE,M2_OUT_DAYS,M2_INC_DAYS,M2_BUNDLE_REVENUE,M3_TTL_OUT_CALLS,M3_TTL_OUT_MINUTES,M3_TTL_OUT_REVENUE,M3_V_CALLS_TO_FIXED_LINES,M3_V_MINUTES_TO_FIXED_LINES,M3_V_REVENUE_TO_FIXED_LINES,M3_V_CALLS_TO_COMPETITION,M3_V_MINUTES_TO_COMPETITION,M3_V_REVENUE_TO_COMPETITION,M3_V_CALLS_TO_INTERNATIONAL,M3_V_MINUTES_TO_INTERNATIONAL,M3_V_REVENUE_TO_INTERNATIONAL,M3_GPRS_SESSION,M3_GPRS_REVENUE,M3_GPRS_VOLUME,M3_RECHARGES_NUMBER,M3_RECHARGES_VALUE,M3_OUT_DAYS,M3_INC_DAYS,M3_BUNDLE_REVENUE,M4_TTL_OUT_CALLS,M4_TTL_OUT_MINUTES,M4_TTL_OUT_REVENUE,M4_V_CALLS_TO_FIXED_LINES,M4_V_MINUTES_TO_FIXED_LINES,M4_V_REVENUE_TO_FIXED_LINES,M4_V_CALLS_TO_COMPETITION,M4_V_MINUTES_TO_COMPETITION,M4_V_REVENUE_TO_COMPETITION,M4_V_CALLS_TO_INTERNATIONAL,M4_V_MINUTES_TO_INTERNATIONAL,M4_V_REVENUE_TO_INTERNATIONAL,M4_GPRS_SESSION,M4_GPRS_REVENUE,M4_GPRS_VOLUME,M4_RECHARGES_NUMBER,M4_RECHARGES_VALUE,M4_OUT_DAYS,M4_INC_DAYS,M4_BUNDLE_REVENUE,M5_TTL_OUT_CALLS,M5_TTL_OUT_MINUTES,M5_TTL_OUT_REVENUE,M5_V_CALLS_TO_FIXED_LINES,M5_V_MINUTES_TO_FIXED_LINES,M5_V_REVENUE_TO_FIXED_LINES,M5_V_CALLS_TO_COMPETITION,M5_V_MINUTES_TO_COMPETITION,M5_V_REVENUE_TO_COMPETITION,M5_V_CALLS_TO_INTERNATIONAL,M5_V_MINUTES_TO_INTERNATIONAL,M5_V_REVENUE_TO_INTERNATIONAL,M5_GPRS_SESSION,M5_GPRS_REVENUE,M5_RECHARGES_NUMBER,M5_RECHARGES_VALUE,M5_GPRS_VOLUME,M5_OUT_DAYS,M5_INC_DAYS,M5_BUNDLE_REVENUE,M6_TTL_OUT_CALLS,M6_TTL_OUT_MINUTES,M6_TTL_OUT_REVENUE,M6_V_CALLS_TO_FIXED_LINES,M6_V_MINUTES_TO_FIXED_LINES,M6_V_REVENUE_TO_FIXED_LINES,M6_V_CALLS_TO_COMPETITION,M6_V_MINUTES_TO_COMPETITION,M6_V_REVENUE_TO_COMPETITION,M6_V_CALLS_TO_INTERNATIONAL,M6_V_MINUTES_TO_INTERNATIONAL,M6_V_REVENUE_TO_INTERNATIONAL,M6_GPRS_SESSION,M6_GPRS_REVENUE,M6_GPRS_VOLUME,M6_RECHARGES_NUMBER,M6_RECHARGES_VALUE,M6_OUT_DAYS,M6_INC_DAYS,M6_BUNDLE_REVENUE,TARIFF_PLAN,CONNECTION_DAY,SMARTPHONE_FLAG,INSERTED,M1_MINUTES_PER_CALL,M2_MINUTES_PER_CALL,M3_MINUTES_PER_CALL,M4_MINUTES_PER_CALL,M5_MINUTES_PER_CALL,M6_MINUTES_PER_CALL,TENURE_MONTHS,CUSTOMER_TENURE,M1_ARPU,M2_ARPU,M3_ARPU,M4_ARPU,M5_ARPU,M6_ARPU,M1_ROC_TTL_OUT_CALLS,M1_ROC_TTL_OUT_MINUTES,M1_ROC_TTL_OUT_REVENUE,M1_ROC_V_CALLS_TO_FIXED_LINES,M1_ROC_V_MINUTES_TO_FIXED_LINES,M1_ROC_V_REVENUE_TO_FIXED_LINES,M1_ROC_V_CALLS_TO_COMPETITION,M1_ROC_V_MINUTES_TO_COMPETITION,M1_ROC_V_REVENUE_TO_COMPETITION,M1_ROC_V_CALLS_TO_INTERNATIONAL,M1_ROC_V_MINUTES_TO_INTERNATIONAL,M1_ROC_V_REVENUE_TO_INTERNATIONAL,M1_ROC_GPRS_SESSION,M1_ROC_GPRS_REVENUE,M1_ROC_GPRS_VOLUME,M1_ROC_RECHARGES_NUMBER,M1_ROC_RECHARGES_VALUE,M1_ROC_OUT_DAYS,M1_ROC_INC_DAYS,M1_ROC_BUNDLE_REVENUE,M1_ROC_MINUTES_PER_CALL,M1_ROC_ARPU,M13_AVG_TTL_OUT_CALLS,M46_AVG_TTL_OUT_CALLS,M16_AVG_TTL_OUT_CALLS,M13_AVG_TTL_OUT_MINUTES,M46_AVG_TTL_OUT_MINUTES,M16_AVG_TTL_OUT_MINUTES,M13_AVG_TTL_OUT_REVENUE,M46_AVG_TTL_OUT_REVENUE,M16_AVG_TTL_OUT_REVENUE,M13_AVG_V_CALLS_TO_FIXED_LINES,M46_AVG_V_CALLS_TO_FIXED_LINES,M16_AVG_V_CALLS_TO_FIXED_LINES,M13_AVG_V_MINUTES_TO_FIXED_LINES,M46_AVG_V_MINUTES_TO_FIXED_LINES,M16_AVG_V_MINUTES_TO_FIXED_LINES,M13_AVG_V_REVENUE_TO_FIXED_LINES,M46_AVG_V_REVENUE_TO_FIXED_LINES,M16_AVG_V_REVENUE_TO_FIXED_LINES,M13_AVG_V_CALLS_TO_COMPETITION,M46_AVG_V_CALLS_TO_COMPETITION,M16_AVG_V_CALLS_TO